In [1]:
!pip install qulacs

In [8]:
import numpy as np
from qulacs import ParametricQuantumCircuit, QuantumState
from qulacs.state import inner_product

データセットの生成

In [39]:
num_each_dataset = 2

c_plus = (np.pi/4 + np.pi/4) * np.random.rand(num_each_dataset, 2*n) - np.pi/4 # - \pi/4 ～ \pi/4 のランダムな数
c_minus = (np.pi/4 + np.pi/4) * np.random.rand(num_each_dataset, 2*n) - np.pi/4
y_plus = np.ones(num_each_dataset)
y_minus = -np.ones(num_each_dataset)
c = np.concatenate([c_plus, c_minus])
y = np.concatenate([y_plus, y_minus])

print(c)
print(y)

[[ 0.60774507  0.71037461  0.71960703  0.59213963 -0.12379604 -0.41246352
  -0.22829437 -0.03593825 -0.46469295  0.19432748]
 [ 0.62528531 -0.55958582  0.38193461 -0.75195003  0.31786811  0.42451572
  -0.58566798  0.63189248 -0.67445755 -0.52665039]
 [ 0.17511065  0.61055576 -0.38091255  0.32551635  0.31488296 -0.07852846
  -0.27076839  0.61065792 -0.14819699  0.572839  ]
 [-0.60744946 -0.26154769 -0.4846034   0.58733831  0.39949007 -0.58804688
   0.71642451 -0.54500132  0.62243523  0.41255036]]
[ 1.  1. -1. -1.]


回路の作成

In [42]:
# D_x V_\lambda |0> を生成する回路
def kernel_circuit(c):
    circuit = ParametricQuantumCircuit(num_qubits)
    # V
    for i in range(num_qubits):
        circuit.add_parametric_RY_gate(i,angle)
    # T型のトポロジー
    circuit.add_CZ_gate(0,1)
    circuit.add_CZ_gate(1,2)
    circuit.add_CZ_gate(1,3)
    circuit.add_CZ_gate(3,4)

    # D
    for i in range(num_qubits):
        circuit.add_parametric_RZ_gate(i,c[2*i])
        circuit.add_parametric_RX_gate(i,c[2*i+1])
    
    return circuit

# グラム行列の要素の計算
def kernel_entry(c1, c2):
    state1 = QuantumState(num_qubits)
    state2 = QuantumState(n)
    
    state1.set_zero_state()
    state2.set_zero_state()
    
    circuit1 = kernel_circuit(c1)
    circuit2 = kernel_circuit(c2)
    
    circuit1.update_quantum_state(state1)
    circuit2.update_quantum_state(state2)
    
    return abs(inner_product(state1, state2)) ** 2

グラム行列の計算

In [43]:
num_qubits = 5
angle = 0.1 # np.pi/2が最適値？

gram_matrix = np.zeros([2*num_each_dataset, 2*num_each_dataset])

# グラム行列計算（計算量減らせる -> 後で行う）
for i in range(2*num_each_dataset):
    for j in range(2*num_each_dataset):
        kernel = kernel_entry(c[i],c[j])
        gram_matrix[i,j] = kernel
        
print(gram_matrix)

[[1.         0.25050696 0.80223347 0.68015535]
 [0.25050696 1.         0.36727245 0.27536201]
 [0.80223347 0.36727245 1.         0.54365081]
 [0.68015535 0.27536201 0.54365081 1.        ]]
